<title>Capstone Project</title>

<title>Capstone - Data Science</title>

<h1>Analysis of Neighborhoods and Boroughs in Toronto, Canada for trending venues.</h1>

<h3>IBM Data Science Professional - Coursera Specialization</h3>

<h4>Import <i>Pandas</i> and <i>Numpy</i> libraries</h4>

In [1]:
import pandas as pd
import numpy as np

<h2> Part 1: Data Scraping and Wrangling</h2>

<h4>Install the libraries needed for the web scraping exercise</h4>

In [2]:
!pip install requests
!pip install bs4
!pip install plotly

<h4>Import the libraries needed for the web scraping exercise</h4>

In [3]:
import requests
from bs4 import BeautifulSoup

<h4>Utilize the <i>requests</i> library to download the Wikipedia Page</h4>

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url)
html_data=response.content

<h4>Parse the html using <i>beautiful soup</i></h4>

In [5]:
soup = BeautifulSoup(html_data)

<h4>Create a list to store table data</h4>

In [6]:
table_contents=[]

<h4>Extract the table from the html containing the attributes: PostalCode, Borough and Neighborhood</h4>

In [7]:
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

<h4>Store the extracted table contents into a dataframe</h4>

In [8]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

<h4>Display the first 5 rows of the dataframe to check for values</h4>

In [9]:
df.head()

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Queen's Park,Ontario Provincial Government,M7A


In [10]:
df.shape

(103, 3)

<h2>Part 2: Geospatial Analysis</h2>

<h3>2.1: Determine the longitude and latitude of the Postal Codes</h3>

<h3>Install and import the <i>Geocoder</i> python package</h3>

In [11]:
!pip install geocoder

In [12]:
import geocoder

<h4>Initialize the Geo variables to none</h4>

In [13]:
lat_lng_coords = None

<h4>Loop until we obtain the Geo coordinates</h4>

In [14]:
#Due to common errors with this package, a CSV file with the values pre-defined will be utilized instead
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario')
#  lat_lng_coords = g.latlng
#
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

<h4>Utilizing a CSV file with the files pre-defined</h4>

In [15]:
Geo_file = 'C:\\Users\\hayth\\Downloads\\Geospatial_Coordinates.csv'
lat_lng_coord=pd.read_csv(Geo_file)
lat_lng_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h4>Add the two columns to the original data frame</h4>

In [16]:
lat_lng_coord.rename(columns = {"Postal Code": "PostalCode"}, inplace=True) #Rename column in the Data Frame to match the naming with the original data frame and allow for the merging of the two data frames
main_df = pd.merge(df, lat_lng_coord, on='PostalCode')
main_df.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [17]:
main_df.shape

(103, 5)

<h3>2.2: Connect to the Foursquare API</h3>

<h4>Note: This analysis replicate the analysis performed to the New York Data by selecting a specific location (borough) and exploring venues around the selected location.</h4>

<h4>Declare the Foursquare credentials and variables</h4>

In [18]:
CLIENT_ID = 'ZTLKNZCELC33KXINJUOXG2EDYQSGTRR5JPETP1DF4OTI3RMU' #Foursquare ID
CLIENT_SECRET = 'A5MSF4DLXAIEPYAJH5VKC52BPHFPLJ03UC4BJBVLVIRTMTSW' #Foursquare Secret
ACCESS_TOKEN = 'L33AUJG4MP4DRH0IUJNUNUFKZFEMCOEQ3QM4153GL1O0SAJK' #FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
radius = 500

<h3>2.3: Selecting Downtown Toronto borough in the Toronto dataframe and exploring venues around its location.</h3>

<h4>Select one of the boroughs in our Toronto dataframe</h4>

In [19]:
main_df.loc[2, 'Borough']

'Downtown Toronto'

<h4>Determine the latitude and longitude values of the "Downtown Toronto" borough</h4>

In [20]:
latitude = main_df.loc[2, 'Latitude']
longitude = main_df.loc[2, 'Longitude']
print("The latitude and longitude of {} borough are respectively {} and {}.".format(main_df.loc[2, 'Borough'],
                                                                                    latitude,
                                                                                    longitude))

The latitude and longitude of Downtown Toronto borough are respectively 43.6542599 and -79.3606359.


In [21]:
#in case we need to specify a specific search query, we input our search parameter into this object and pass it to the below search URL
search = ''

<h4>Declare the Foursquare search URL as an object</h4>

In [22]:
search_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)

<h4>Receive the GET Request and examine the results from the declared URL</h4>

In [23]:
results = requests.get(search_url).json()
results

{'meta': {'code': 200, 'requestId': '60f6f5edf180a40a3d4bdc0e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '5bdc6c2bba57b4002c4c71a8',
    'name': 'Oldtown Bodega',
    'location': {'lat': 43.653966,
     'lng': -79.360752,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.653966,
       'lng': -79.360752}],
     'distance': 34,
     'postalCode': 'M5A 1L6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Toronto ON M5A 1L6', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1626797549',
    'hasPerk': False},
   {'id': '4dc9d4d9d16495ca5add0803',
    'name': "Cam's Auto Service",
    'location': {'addr

<h4>The below function streamlines the process for getting the relevant part of the received JSON for any neighborhoud</h4>

In [24]:
def get_venues(neighborhood, lat, long, limit=100, radius=250):
    params = dict(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        v=VERSION,
        ll=f"{lat}, {long}",
        radius=radius,
        limit=limit
    )
    venues = []
    data = requests.get(url=search_url, params=params).json()
    for v in data['response']['venues']:
        venues.append([neighborhood, lat, long, v['name'], v['location']['lat'], v['location']['lng']])
    return venues

In [25]:
#get_venues('North York', 43.7532586, -79.3296565)

<h4>Obtain the relevant part of the JSON of the declared URL</h4>

In [26]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

<h4>Transform the relevant JSON venues list into a dataframe</h4>

In [27]:
# import the library that allows tranforming json files into a pandas dataframe library
from pandas.io.json import json_normalize


# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,5bdc6c2bba57b4002c4c71a8,NaN,CA,Toronto,Canada,NaN,34,"[Toronto ON M5A 1L6, Canada]","[{'label': 'display', 'lat': 43.653966, 'lng':...",43.653966,-79.360752,M5A 1L6,ON,Oldtown Bodega,v-1626797549,NaN
1,"[{'id': '4bf58dd8d48988d124951735', 'name': 'A...",False,4dc9d4d9d16495ca5add0803,475 King Street East,CA,Toronto,Canada,Sackville Street,10,"[475 King Street East (Sackville Street), Toro...","[{'label': 'display', 'lat': 43.65419500779484...",43.654195,-79.360545,M4M 2T7,ON,Cam's Auto Service,v-1626797549,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,53b8466a498e83df908c3f21,368 King St E,CA,Toronto,Canada,at Trinity St,122,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65355870959944...",43.653559,-79.361809,NaN,ON,Tandem Coffee,v-1626797549,NaN
3,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,5e3618557bde930007d18b8d,501 King St E,CA,Toronto,Canada,NaN,103,"[501 King St E, Toronto ON M5A 1L9, Canada]","[{'label': 'display', 'lat': 43.65481, 'lng': ...",43.654810,-79.359595,M5A 1L9,ON,Gusto 501,v-1626797549,NaN
4,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4d067a2b26adb1f702f9dc70,475 King St. E,CA,Toronto,Canada,NaN,17,"[475 King St. E, Toronto ON M5A 1L6, Canada]","[{'label': 'display', 'lat': 43.65424167246142...",43.654242,-79.360853,M5A 1L6,ON,Thruway Muffler / Downtown Brake,v-1626797549,NaN


<h4>Filter dataframe according to informaiton of interest.</h4>

In [28]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Oldtown Bodega,Café,NaN,CA,Toronto,Canada,NaN,34,"[Toronto ON M5A 1L6, Canada]","[{'label': 'display', 'lat': 43.653966, 'lng':...",43.653966,-79.360752,M5A 1L6,ON,5bdc6c2bba57b4002c4c71a8
1,Cam's Auto Service,Automotive Shop,475 King Street East,CA,Toronto,Canada,Sackville Street,10,"[475 King Street East (Sackville Street), Toro...","[{'label': 'display', 'lat': 43.65419500779484...",43.654195,-79.360545,M4M 2T7,ON,4dc9d4d9d16495ca5add0803
2,Tandem Coffee,Coffee Shop,368 King St E,CA,Toronto,Canada,at Trinity St,122,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65355870959944...",43.653559,-79.361809,NaN,ON,53b8466a498e83df908c3f21
3,Gusto 501,Italian Restaurant,501 King St E,CA,Toronto,Canada,NaN,103,"[501 King St E, Toronto ON M5A 1L9, Canada]","[{'label': 'display', 'lat': 43.65481, 'lng': ...",43.654810,-79.359595,M5A 1L9,ON,5e3618557bde930007d18b8d
4,Thruway Muffler / Downtown Brake,Building,475 King St. E,CA,Toronto,Canada,NaN,17,"[475 King St. E, Toronto ON M5A 1L6, Canada]","[{'label': 'display', 'lat': 43.65424167246142...",43.654242,-79.360853,M5A 1L6,ON,4d067a2b26adb1f702f9dc70


<h4>Create a new dataframe showing the data we only need (further filtering)</h4>

In [29]:
main_filtered = dataframe_filtered.drop(['country', 'crossStreet', 'labeledLatLngs', 'state', 'id'], axis=1)
main_filtered.head()

,name,categories,address,cc,city,distance,formattedAddress,lat,lng,postalCode
0,Oldtown Bodega,Café,NaN,CA,Toronto,34,"[Toronto ON M5A 1L6, Canada]",43.653966,-79.360752,M5A 1L6
1,Cam's Auto Service,Automotive Shop,475 King Street East,CA,Toronto,10,"[475 King Street East (Sackville Street), Toro...",43.654195,-79.360545,M4M 2T7
2,Tandem Coffee,Coffee Shop,368 King St E,CA,Toronto,122,"[368 King St E (at Trinity St), Toronto ON, Ca...",43.653559,-79.361809,NaN
3,Gusto 501,Italian Restaurant,501 King St E,CA,Toronto,103,"[501 King St E, Toronto ON M5A 1L9, Canada]",43.654810,-79.359595,M5A 1L9
4,Thruway Muffler / Downtown Brake,Building,475 King St. E,CA,Toronto,17,"[475 King St. E, Toronto ON M5A 1L6, Canada]",43.654242,-79.360853,M5A 1L6


<h4>Since we have limited number of request quota on Foursquare free tier, we will save the dataframe to a csv file, so that we can continue from here at a later point in time, if we need to close the notebook.

In [30]:
main_filtered.to_csv('toronto_boroughs_and_venues.csv')

<h3>2.4: Explore and visualize the venue data on a map (GIS)

<h4>Read the CSV and drop unnamed entries

In [31]:
main_T = pd.read_csv('toronto_boroughs_and_venues.csv')
main_T.drop(['Unnamed: 0'], axis=1, inplace=True)
main_T.head()

,name,categories,address,cc,city,distance,formattedAddress,lat,lng,postalCode
0,Oldtown Bodega,Café,NaN,CA,Toronto,34,"['Toronto ON M5A 1L6', 'Canada']",43.653966,-79.360752,M5A 1L6
1,Cam's Auto Service,Automotive Shop,475 King Street East,CA,Toronto,10,"['475 King Street East (Sackville Street)', 'T...",43.654195,-79.360545,M4M 2T7
2,Tandem Coffee,Coffee Shop,368 King St E,CA,Toronto,122,"['368 King St E (at Trinity St)', 'Toronto ON'...",43.653559,-79.361809,NaN
3,Gusto 501,Italian Restaurant,501 King St E,CA,Toronto,103,"['501 King St E', 'Toronto ON M5A 1L9', 'Canada']",43.654810,-79.359595,M5A 1L9
4,Thruway Muffler / Downtown Brake,Building,475 King St. E,CA,Toronto,17,"['475 King St. E', 'Toronto ON M5A 1L6', 'Cana...",43.654242,-79.360853,M5A 1L6


In [32]:
#If we need to Keep only the rows with Toronto as city
main_T[main_T['city'] == 'Toronto']

,name,categories,address,cc,city,distance,formattedAddress,lat,lng,postalCode
0,Oldtown Bodega,Café,NaN,CA,Toronto,34,"['Toronto ON M5A 1L6', 'Canada']",43.653966,-79.360752,M5A 1L6
1,Cam's Auto Service,Automotive Shop,475 King Street East,CA,Toronto,10,"['475 King Street East (Sackville Street)', 'T...",43.654195,-79.360545,M4M 2T7
2,Tandem Coffee,Coffee Shop,368 King St E,CA,Toronto,122,"['368 King St E (at Trinity St)', 'Toronto ON'...",43.653559,-79.361809,NaN
3,Gusto 501,Italian Restaurant,501 King St E,CA,Toronto,103,"['501 King St E', 'Toronto ON M5A 1L9', 'Canada']",43.654810,-79.359595,M5A 1L9
4,Thruway Muffler / Downtown Brake,Building,475 King St. E,CA,Toronto,17,"['475 King St. E', 'Toronto ON M5A 1L6', 'Cana...",43.654242,-79.360853,M5A 1L6
5,Libbys Custom Framing,Arts & Crafts Store,465 King Street,CA,Toronto,15,"['465 King Street', 'Toronto ON M5A 1L6', 'Can...",43.654126,-79.360682,M5A 1L6
6,Hastings Barbershop Corktown,Salon / Barbershop,NaN,CA,Toronto,32,"['Toronto ON M5A 1L4', 'Canada']",43.654175,-79.361020,M5A 1L4
7,Bottē Gallery,Art Gallery,398 King st East,CA,Toronto,32,"['398 King st East', 'Toronto ON M5A 1K9', 'Ca...",43.654110,-79.360976,M5A 1K9
8,King Deli Cafe,Sandwich Place,465 King Street E,CA,Toronto,56,['465 King Street E (just East of Parliament S...,43.653906,-79.361147,NaN
9,Sackville Playground,Park,420 king st E,CA,Toronto,75,"['420 king st E', 'Toronto ON', 'Canada']",43.654656,-79.359871,NaN


<h4>Visualize the findings on a map

In [33]:
! pip install folium==0.5.0
import folium # plotting library

print('Folium installed and imported')

Folium installed and imported


In [34]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=20) # generate map centred around Downtown Toronto

# add Downtown Toronto as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Downtown Toronto',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(main_T.lat, main_T.lng, main_T.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)
    
# display map
venues_map

<h4>Instead of just exploring the venues around the defined location (Downtown Toronto), we want to explore the TRENDING VENUES around this location. Let's check for food venues.

<h4>Define a new URL to the Foursquare API with the parameter "trending"

In [35]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60f6f5f32222750a72e41e96'},
 'response': {'venues': []}}

<h4>Check if any venues are trending at this time

In [36]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

<h4>Visualize the trending venues

In [37]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=20) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Downtown Toronto',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

<h4>As can be seen, no trending venues are present at Downtown Toronto at this moment.

<h2>Part 3: Clustering via Machine Learning

There are many models for clustering. We will be utilizing <i>k-means</i> as it is vastly used for clustering in many data science applications, especially useful if we need to quickly discover insights from unlabeled data.

<h4>Import the <i>sklearn</i> library to import the <i>kmeans</i> library

In [38]:
from sklearn.cluster import KMeans

<h4>Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h4>Utilize the abovementioned function

In [40]:
toronto_venues = getNearbyVenues(names=main_df['Neighborhood'],
                                   latitudes=main_df['Latitude'],
                                   longitudes=main_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

<h4>Let's check how many venues were returned for each neighborhood

In [41]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,30,30,30,30,30,30
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14


<h4>Determine the number of unique categories can be curated from all the returned venues

In [42]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


<h4>Analyze each neighborhood individually

In [43]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

<h4>Examine the new dataframe size.

In [44]:
toronto_onehot.shape

(1337, 237)

<h4>Next, group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [45]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

<h4>Print each neighborhood along with the top 5 most common venues

In [46]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2             Clothing Store   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1  Sandwich Place  0.14
2             Gym  0.14
3             Pub  0.14
4     Coffee Shop  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2                Pizza Place  0.05
3                      Diner  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1        Sandwich Place

<h4>Upload new results into a pandas dataframe

<h4>A function to sort the venues in descending order.

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h4>Create the new dataframe and display the top 10 venues for each neighborhood.

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Clothing Store,Latin American Restaurant,Skating Rink,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Pub,Skating Rink,Sandwich Place,Coffee Shop,Dog Run,Donut Shop,Distribution Center,Dance Studio
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pizza Place,Deli / Bodega,Sushi Restaurant,Supermarket,Bridal Shop,Diner,Shopping Mall,Sandwich Place
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Pharmacy,Pub,Juice Bar,Butcher,Fast Food Restaurant,Café,Indian Restaurant


<h4>Run <i>k-means</i> to cluster the neighborhood into 9 clusters.

In [49]:
# set number of clusters
kclusters = 9

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 3, 6, 6, 6, 6, 6, 6, 6, 0])

<h4>Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = main_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,M3A,43.753259,-79.329656,0.0,Bus Stop,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
1,North York,Victoria Village,M4A,43.725882,-79.315572,6.0,Intersection,Portuguese Restaurant,Coffee Shop,French Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,6.0,Coffee Shop,Bakery,Park,Breakfast Spot,Gym / Fitness Center,Historic Site,Restaurant,Pub,Performing Arts Venue,Mexican Restaurant
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763,6.0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Boutique,Vietnamese Restaurant,Coffee Shop,Distribution Center,Dim Sum Restaurant,Diner
4,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494,6.0,Coffee Shop,Sushi Restaurant,Gym,Hobby Shop,Bar,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place


<h4>Import plotting libraries

In [53]:
#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

<h4>Finally visualize the resulting clusters

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4>Examine the clusters obtained individually

<h4>Cluster 1

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,0.0,Bus Stop,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
16,Humewood-Cedarvale,0.0,Field,Park,Trail,Hockey Arena,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
21,Caledonia-Fairbanks,0.0,Park,Women's Store,Pool,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
35,The Danforth East,0.0,Coffee Shop,Park,Convenience Store,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
40,Downsview East,0.0,Airport,Park,Women's Store,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
61,Lawrence Park,0.0,Park,Swim School,Dim Sum Restaurant,Bus Line,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
66,York Mills West,0.0,Park,Convenience Store,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
83,"Moore Park, Summerhill East",0.0,Lawyer,Park,Tennis Court,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
91,Rosedale,0.0,Park,Playground,Trail,Curling Ice,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


<h4> Cluster 2

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Glencairn,1.0,Playground,Park,Bakery,Japanese Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
32,Scarborough Village,1.0,Playground,Spa,College Cafeteria,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
85,"Milliken, Agincourt North, Steeles East, L'Amo...",1.0,Playground,Intersection,Park,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


<h4> Cluster 3

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Humber Summit,2.0,Gym,Pizza Place,Home Service,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
62,Roselawn,2.0,Home Service,Garden,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


<h4> Cluster 4

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Christie,3.0,Grocery Store,Café,Park,Athletics & Sports,Baby Store,Candy Store,Restaurant,Italian Restaurant,Nightclub,Coffee Shop
29,Thorncliffe Park,3.0,Indian Restaurant,Sandwich Place,Yoga Studio,Bank,Grocery Store,Gym,Gas Station,Fast Food Restaurant,Liquor Store,Middle Eastern Restaurant
46,Downsview West,3.0,Grocery Store,Shopping Mall,Park,Bank,Hotel,Women's Store,Diner,Dessert Shop,Dim Sum Restaurant,Distribution Center
47,"India Bazaar, The Beaches West",3.0,Pizza Place,Park,Sandwich Place,Sushi Restaurant,Italian Restaurant,Fish & Chips Shop,Brewery,Ice Cream Shop,Restaurant,Food & Drink Shop
56,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",3.0,Sandwich Place,Skating Rink,Discount Store,Turkish Restaurant,Women's Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
60,Downsview Northwest,3.0,Gym / Fitness Center,Athletics & Sports,Liquor Store,Discount Store,Grocery Store,Airport Food Court,Colombian Restaurant,Falafel Restaurant,Event Space,Escape Room
63,"Runnymede, The Junction North",3.0,Grocery Store,Pizza Place,Brewery,Convenience Store,Bus Line,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
65,"Dorset Park, Wexford Heights, Scarborough Town...",3.0,Indian Restaurant,Chinese Restaurant,Pet Store,Vietnamese Restaurant,Brewery,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
67,Davisville North,3.0,Gym / Fitness Center,Hotel,Breakfast Spot,Food & Drink Shop,Department Store,Sandwich Place,Park,Concert Hall,Dessert Shop,Eastern European Restaurant
70,Westmount,3.0,Playground,Chinese Restaurant,Discount Store,Middle Eastern Restaurant,Intersection,Sandwich Place,Pizza Place,Coffee Shop,Deli / Bodega,Drugstore


<h4> Cluster 5

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Malvern, Rouge",4.0,Fast Food Restaurant,Women's Store,Falafel Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


<h4> Cluster 6

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,"York Mills, Silver Hills",5.0,Park,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
52,"Willowdale, Newtonbrook",5.0,Park,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


<h4> Cluster 7

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,6.0,Intersection,Portuguese Restaurant,Coffee Shop,French Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
2,"Regent Park, Harbourfront",6.0,Coffee Shop,Bakery,Park,Breakfast Spot,Gym / Fitness Center,Historic Site,Restaurant,Pub,Performing Arts Venue,Mexican Restaurant
3,"Lawrence Manor, Lawrence Heights",6.0,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Boutique,Vietnamese Restaurant,Coffee Shop,Distribution Center,Dim Sum Restaurant,Diner
4,Ontario Provincial Government,6.0,Coffee Shop,Sushi Restaurant,Gym,Hobby Shop,Bar,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place
7,Don Mills North,6.0,Gym,Caribbean Restaurant,Dessert Shop,Japanese Restaurant,Café,Concert Hall,Department Store,Electronics Store,Eastern European Restaurant,Drugstore
8,"Parkview Hill, Woodbine Gardens",6.0,Pizza Place,Gym / Fitness Center,Athletics & Sports,Café,Breakfast Spot,Flea Market,Bank,Intersection,Pharmacy,Gastropub
9,"Garden District, Ryerson",6.0,Café,Clothing Store,Ramen Restaurant,Pizza Place,Burger Joint,Japanese Restaurant,Sporting Goods Shop,Electronics Store,New American Restaurant,Bookstore
11,"West Deane Park, Princess Gardens, Martin Grov...",6.0,Bakery,Middle Eastern Restaurant,Women's Store,Falafel Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
12,"Rouge Hill, Port Union, Highland Creek",6.0,Construction & Landscaping,Bar,Women's Store,Dessert Shop,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
13,Don Mills South,6.0,Gym,Restaurant,Coffee Shop,Bike Shop,Art Gallery,Dim Sum Restaurant,Supermarket,Sandwich Place,Discount Store,Asian Restaurant


<h4> Cluster 8

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Downsview Central,7.0,Baseball Field,Food Truck,Women's Store,Falafel Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
57,"Humberlea, Emery",7.0,Baseball Field,Women's Store,Farmers Market,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",7.0,Baseball Field,Women's Store,Farmers Market,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


<h4> Cluster 9

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Weston,8.0,Convenience Store,Women's Store,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


<h4>Visualize the findings of Ashrafieh Zone Number: 62 on a map

In [88]:
venues_map_1 = folium.Map(location=[latitude_1, longitude_1], zoom_start=20) # generate map centred around Ashrafieh

# add Ashrafieh Zone Number:62 as a red circle mark
folium.CircleMarker(
    [latitude_1, longitude_1],
    radius=10,
    popup='Ashrafieh Zone Number: 62',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map_1)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(main_filtered_1.lat, main_filtered_1.lng, main_filtered_1.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map_1)
    
# display map
venues_map_1